# Assignment 6: Multiple start local search (MSLS) and iterated local search (ILS)

- Łukasz Andryszewski 151930
- Filip Firkowski 151946

Link to the repository is: https://github.com/lucapl/Evolutionary-Computations.

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd

from utils import *
from plotting import *
pd.set_option('display.max_colwidth', None)

## Description of a problem:

The problem is about selecting exactly 50% of the nodes to form a Hamiltonian cycle that minimizes the total distance of the path and the total cost of the selected nodes.

In this report the candidate edges mechanism is evaluated. The goal being to speed up the steepest local search algorithm.

## Pseudocode of all implemented algorithms

<style>
  .no-page-break {
    page-break-inside: avoid;
  }
</style>

<div class="no-page-break">
  <h3>Multiple Start Local Search (MSLS):</h3>
  <pre>
function MultipleStartLocalSearch():
  bestSolution = null
  bestObjectiveValue = Infinity

  for i = 1 to numberOfStarts:
    initialSolution = createSolver().solve(randomStartingCity())
    localSolution = localSearch(initialSolution)

    currentObjectiveValue = localSolution.getObjectiveFunctionValue()
    if currentObjectiveValue < bestObjectiveValue:
      bestObjectiveValue = currentObjectiveValue
      bestSolution = localSolution

  return bestSolution

function localSearch(solution):
  moveset = createMoveset()
  bestSolution = solution

  for i = 1 to MAX_LOCAL_SEARCH_ITERATIONS:
    bestMove = null
    bestMoveCost = Infinity

    for each move in moveset:
      moveCost = evaluateMove(solution, move)
      if moveCost < bestMoveCost:
        bestMoveCost = moveCost
        bestMove = move

    if bestMove is not null:
      solution.performMove(bestMove)
      
  return solution
  </pre>
</div>

<div class="no-page-break">
  <h3>Iterated Local Search (ILS):</h3>
  <pre>
function IteratedLocalSearch():
  initialSolution = createSolver().solve(randomStartingCity())
  bestSolution = localSearch(initialSolution)
  bestObjectiveValue = bestSolution.getObjectiveFunctionValue()

  for iteration = 1 to maxIterations:
    perturbedSolution = perturb(bestSolution)
    newSolution = localSearch(perturbedSolution)

    newObjectiveValue = newSolution.getObjectiveFunctionValue()
    if newObjectiveValue < bestObjectiveValue:
      bestSolution = newSolution
      bestObjectiveValue = newObjectiveValue
  
  return bestSolution

function perturb(solution):
  perturbedSolution = copy(solution)
  applyRandomSwapOrInversion(perturbedSolution)

  return perturbedSolution
  </pre>
</div>


<style>
  table {
    width: 100%;
    table-layout: fixed;
    word-wrap: break-word;
  }
</style>

## Results of a computational experiments

In [1]:
solver_types = ["msls", "ils"]
instances = ['A', 'B']

folder_path = '../out6'
all_json_data = load_json_files_from_folder(folder_path)

table, best_solutions = get_best_solutions_and_vertical_table(solver_types,instances,all_json_data)

NameError: name 'load_json_files_from_folder' is not defined

In [ ]:
localSearch = read_html('<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Method</th>\n      <th>Instance A</th>\n      <th>Instance B</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>localSearchSteepestNodesRandom</td>\n      <td>88106.7 (79865.0-98435.0)</td>\n      <td>63238.6 (56390.0-72239.0)</td>\n    </tr>\n    <tr>\n      <td>localSearchSteepestEdgesRandom</td>\n      <td>74001.6 (71311.0-77763.0)</td>\n      <td>48249.9 (45981.0-51599.0)</td>\n    </tr>\n  </tbody>\n</table>')

display_html(pd.concat([localSearch,table]),False)

<p style="page-break-after:always;"></p>

Table containing elapsed time in ms.

In [ ]:
timeTable, _ = get_best_solutions_and_vertical_table(solver_types,instances,all_json_data,"elapsed time")
localSearchTime = read_html('<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Method</th>\n      <th>Instance A</th>\n      <th>Instance B</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>localSearchSteepestNodesRandom</td>\n      <td>172.0 (136.8-430.3)</td>\n      <td>169.2 (129.4-273.2)</td>\n    </tr>\n    <tr>\n      <td>localSearchSteepestEdgesRandom</td>\n      <td>118.9 (104.8-145.0)</td>\n      <td>139.6 (105.2-496.6)</td>\n    </tr>\n  </tbody>\n</table>')

display_html(pd.concat([localSearchTime,timeTable]),False)

## Best solutions:

The following solutions were checked with the solution checker.

In [ ]:
print_solutions(solver_types,instances,best_solutions)

## 2D visualizations:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

coordinates = {
    'A': load_coordinates_from_csv('../src/main/resources/instances/TSPA.csv'),
    'B': load_coordinates_from_csv('../src/main/resources/instances/TSPB.csv')
}

plot_all(solver_types, instances, coordinates, best_solutions)

<p style="page-break-after:always;"></p>

# Conclusions:

Multiple Start Local Search (MSLS) enhances solution quality by performing multiple independent local searches from different starting points, increasing the chances of avoiding local minima. However, it can be computationally expensive as the number of starts grows.

Iterated Local Search (ILS) improves a single solution through iterative perturbations and local searches, striking a balance between exploration and exploitation. The perturbation size and iteration count are key factors for optimal performance.

Both MSLS and ILS aim to overcome local minima issues, with MSLS being more effective for diverse solutions and ILS excelling in refining a single solution. They complement each other well: MSLS can provide good initial solutions for ILS to improve. Proper tuning of parameters like perturbation size and number of starts is crucial for achieving the best results. 

However, overall ILS performed better because its focused iterative refinement tends to yield higher-quality solutions in fewer iterations compared to MSLS, which requires a larger computational effort. The more concentrated exploration in ILS allowed it to converge more effectively toward optimal solutions.